In [1]:
using Logging, LinearAlgebra, Revise

In [2]:
global_logger(ConsoleLogger(stdout, Logging.Info))
using PorousMaterials, MOFun

[ Info: Precompiling MOFun [top-level]


# Example case: make IRMOF-1_one_ring with 2-acetylamido-terephthalate linker

## parent structure

In [3]:
parent_structure = Crystal("IRMOF-1_one_ring.cif")
strip_numbers_from_atom_labels!(parent_structure)
infer_bonds!(parent_structure, true)

┌ Info: Crystal IRMOF-1_one_ring.cif has  space group. I am converting it to P1 symmetry.
└         To afrain from this, pass `convert_to_p1=false` to the `Crystal` constructor.


## search moiety

In [4]:
s_moty = moiety("find-replace/2-!-p-phenylene");

#### nodes are sorted by order, except that R-group nodes are sent to the end of the list

Sorting the tagged nodes to the end may degrade performance for multi-nuclear replacement motifs.  This is a compromise made for the simplicity of aligning the moiety; it forces the isomorphism between the search moiety alignment mask and the replace moiety to keep a consistent node ordering with the isomorphism between the search moiety and the parent structure.

#### should write a test for tagged atoms being at the end

## replace moiety

In [5]:
r_moty = moiety("2-acetylamido-p-phenylene");

## substructure search results

In [6]:
search = s_moty ∈ parent_structure;

#### all 4 are in 1 location

## choose 1 at random for a replacement

In [7]:
s2p_isomorphism = search.results[rand(1:4)].isomorphism;

## subtract R group to get mask

In [8]:
mask = MOFun.subtract_r_group(s_moty)
infer_bonds!(mask, false)

## align mask to replace moiety

In [9]:
m2r_isomorphism = (mask ∈ r_moty).results[1].isomorphism;

#### only 1 configuration, but will not always be so (heteroleptic multi-functionalization)

## now we have the correspondence between the search moiety and our target location, and between the search moiety mask and the replace moiety.  we can perform orthogonal Procrustes

In [28]:
with_logger(ConsoleLogger(stdout, Logging.Debug)) do
    altered_crystal = MOFun.effect_replacement(parent_structure, s_moty, r_moty, s2p_isomorphism, m2r_isomorphism)
    write_cif(altered_crystal, "output/" * altered_crystal.name * ".cif")
end

┌ Debug: adjust_for_pb
│   xtal.name = "IRMOF-1_one_ring.cif"
└ @ MOFun C:\Users\eahen\.julia\dev\MOFfun.jl\src\findreplace.jl:95
┌ Debug: adjust_for_pb
│   xtal.name = "find-replace/2-!-p-phenylene"
└ @ MOFun C:\Users\eahen\.julia\dev\MOFfun.jl\src\findreplace.jl:95


AssertionError: AssertionError: Invalid xf coords in find-replace/2-!-p-phenylene

In [11]:
s_moty.box

Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 1.000000 Å. b = 1.000000 Å, c = 1.000000 Å
	Volume of unit cell: 1.000000 Å³
